In [21]:
import glob
import nibabel as nb
import numpy as np
import os

from pathlib import Path
from PIL import Image

Here I just want to check whether I am in the correct directory, which in my 
case starts with "2022". The base_dir is assumed to house the training data 
so you might want to change the conditional expression or ignore the output 
altogether.

In [3]:
base_path = Path(os.getcwd())

if "2022" in base_path.stem: 
    print("We are in the correct wd")
else:
    print(f"We are in the wrong wd: {base_path}")

We are in the correct wd


## Functions used for conversion
The first function recursively indexes all PNG files that are to be found in 
the given directory, and the second function converts one image, using the 
information given by Hrvoje Bogunovic. 

In [40]:
def index_files(directory: Path | str): 

    """Returns a list of all PNG files found recursivel in `directory`."""

    if isinstance(directory, Path): 
        directory = str(directory)

    path_pattern = directory + "/**/*.png"
    files = glob.glob(path_pattern)

    path_pattern = directory + "/*.png"
    files.extend(glob.glob(path_pattern))
    return files


def convert_file(fpath: Path | str, x_axis_len = 4, y_axis_len = 4*10**(-3)):
    """ 
    Converts a PNG file `fpath` to a NiFti file.

    For this it is assumed that the length of the x axis in the scanner's 
    coordinate system is 4mm and the y axis 4um. An affine transformation matrix
    is defined that converts voxel coordinates back into the scanner's 
    coordinate system without rotation or translation. 

    NiFti images are given in grayscale. Due to that the PNG image is first 
    converted to grayscale leading to a 2D NiFti image.    

    The scanner's coordinate system has the same origin as the original image's. 
    The only difference being the units on each axis. Where the unit of each 
    axis in the image is one pixel, the unit of the converted image's axes is  
    millimeter. 

    Since the original dimensions are unknown, default lengths for each axis 
    are assumed, with the x axis assumed to be 4 mm in length and the y axis 
    4 um. 

    Parameters:
    -----------
    fpath: Path | str
        Path to the PNG file that shall be converted

    x_axis_len: int (default: 4)
        Metric dimension of the x-axis in the scanner's x axis in mm.

    x_axis_len: int (default: 4e-3)
        Metric dimension of the y-axis in the scanner's y axis in mm.
    """

    if isinstance(fpath, str):
        fpath = Path(fpath)

    dst_fpath = fpath.parent / (fpath.stem + ".nii")

    orig_image = Image.open(fpath).convert('L') # we don't care about the colors, nii images are also grayscale
    img_arr = np.array(orig_image).astype(np.int16)

    # we just want to scale each axis, no rotation and no translation
    x_axis_res, y_axis_res = img_arr.shape
    affine_matrix = np.diag([x_axis_len / x_axis_res, y_axis_len / y_axis_res, 1, 1])

    nii_img = nb.Nifti1Image(img_arr, affine_matrix)
    nii_img.to_filename(str(dst_fpath))

In [42]:
png_images = index_files(base_path / "data" / "Train" / "Image")
for img in png_images: 
    convert_file(img)